In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
import os

py_file_location = "/content/drive/My Drive/photo2cartoon/models"
if py_file_location not in sys.path:
  sys.path.append(os.path.abspath(py_file_location))
py_file_location = "/content/drive/My Drive/photo2cartoon/utils"
if py_file_location not in sys.path:
  sys.path.append(os.path.abspath(py_file_location))
print(sys.path)

['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/My Drive/photo2cartoon/models', '/content/drive/My Drive/photo2cartoon/utils']


In [ ]:
pip install face_alignment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for face-alignment: filename=face_alignment-1.3.5-py2.py3-none-any.whl size=28241 sha256=03bf19e127d87d8068c58326d475c9acf9df6bfde7b3ce4492efa682186c76bc
  Stored in directory: /root/.cache/pip/wheels/c9/ba/4d/2d368f55e5f929f9472da59e356fbdf1483f885de80a5bc620
Successfully built face-alignment


In [ ]:
#pip install networks

In [ ]:
import os
import cv2
import torch
import numpy as np
#from models import ResnetGenerator
from networksw import ResnetGenerator
import argparse
from preprocess import Preprocess


#parser = argparse.ArgumentParser()
#parser.add_argument('/content/drive/MyDrive/photo2cartoon/images', type=str, help='input photo path')
#parser.add_argument('/content/drive/MyDrive/photo2cartoon/images', type=str, help='cartoon save path')
#args = parser.parse_args()

os.makedirs(os.path.dirname('/content/drive/MyDrive/photo2cartoon/images'), exist_ok=True)

class Photo2Cartoon:
    def __init__(self):
        self.pre = Preprocess()
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.net = ResnetGenerator(ngf=32, img_size=256, light=True).to(self.device)
        
        assert os.path.exists('/content/drive/MyDrive/photo2cartoon/models/photo2cartoon_weights.pt'), "[Step1: load weights] Can not find 'photo2cartoon_weights.pt' in folder 'models!!!'"
        params = torch.load('/content/drive/MyDrive/photo2cartoon/models/photo2cartoon_weights.pt', map_location=self.device)
        self.net.load_state_dict(params['genA2B'])
        print('[Step1: load weights] success!')

    def inference(self, img):
        # face alignment and segmentation
        face_rgba = self.pre.process(img)
        if face_rgba is None:
            print('[Step2: face detect] can not detect face!!!')
            return None
        
        print('[Step2: face detect] success!')
        face_rgba = cv2.resize(face_rgba, (256, 256), interpolation=cv2.INTER_AREA)
        face = face_rgba[:, :, :3].copy()
        mask = face_rgba[:, :, 3][:, :, np.newaxis].copy() / 255.
        face = (face*mask + (1-mask)*255) / 127.5 - 1

        face = np.transpose(face[np.newaxis, :, :, :], (0, 3, 1, 2)).astype(np.float32)
        face = torch.from_numpy(face).to(self.device)

        # inference
        with torch.no_grad():
            cartoon = self.net(face)[0][0]

        # post-process
        cartoon = np.transpose(cartoon.cpu().numpy(), (1, 2, 0))
        cartoon = (cartoon + 1) * 127.5
        cartoon = (cartoon * mask + 255 * (1 - mask)).astype(np.uint8)
        cartoon = cv2.cvtColor(cartoon, cv2.COLOR_RGB2BGR)
        print('[Step3: photo to cartoon] success!')
        return cartoon


if __name__ == '__main__':
    img = cv2.cvtColor(cv2.imread('/content/drive/MyDrive/photo2cartoon/images/photo_test.jpg'), cv2.COLOR_BGR2RGB)
    c2p = Photo2Cartoon()
    cartoon = c2p.inference(img)
    if cartoon is not None:
        cv2.imwrite('/content/drive/MyDrive/photo2cartoon/images/test_1.jpg', cartoon)
        print('Cartoon portrait has been saved successfully!')

RuntimeError: ignored

In [ ]:
pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.2 MB 5.2 MB/s 


In [ ]:
import os
import cv2
import numpy as np
import onnxruntime
import argparse
from preprocess import Preprocess


#parser = argparse.ArgumentParser()
#parser.add_argument('--photo_path', type=str, help='input photo path')
#parser.add_argument('--save_path', type=str, help='cartoon save path')
#args = parser.parse_args()

#os.makedirs(os.path.dirname(args.save_path), exist_ok=True)

class Photo2Cartoon:
    def __init__(self):
        self.pre = Preprocess()
        
        assert os.path.exists('/content/drive/MyDrive/photo2cartoon/models/photo2cartoon_weights.onnx'), "[Step1: load weights] Can not find 'photo2cartoon_weights.onnx' in folder 'models!!!'"
        self.session = onnxruntime.InferenceSession('/content/drive/MyDrive/photo2cartoon/models/photo2cartoon_weights.onnx')
        print('[Step1: load weights] success!')

    def inference(self, img):
        # face alignment and segmentation
        face_rgba = self.pre.process(img)
        if face_rgba is None:
            print('[Step2: face detect] can not detect face!!!')
            return None
        
        print('[Step2: face detect] success!')
        face_rgba = cv2.resize(face_rgba, (256, 256), interpolation=cv2.INTER_AREA)
        face = face_rgba[:, :, :3].copy()
        mask = face_rgba[:, :, 3][:, :, np.newaxis].copy() / 255.
        face = (face*mask + (1-mask)*255) / 127.5 - 1

        face = np.transpose(face[np.newaxis, :, :, :], (0, 3, 1, 2)).astype(np.float32)

        # inference
        cartoon = self.session.run(['output'], input_feed={'input':face})

        # post-process
        cartoon = np.transpose(cartoon[0][0], (1, 2, 0))
        cartoon = (cartoon + 1) * 127.5
        cartoon = (cartoon * mask + 255 * (1 - mask)).astype(np.uint8)
        cartoon = cv2.cvtColor(cartoon, cv2.COLOR_RGB2BGR)
        print('[Step3: photo to cartoon] success!')
        return cartoon


if __name__ == '__main__':
    img = cv2.cvtColor(cv2.imread('/content/drive/MyDrive/photo2cartoon/images/test1.jpg'), cv2.COLOR_BGR2RGB)
    c2p = Photo2Cartoon()
    cartoon = c2p.inference(img)
    if cartoon is not None:
        cv2.imwrite('/content/drive/MyDrive/photo2cartoon/images/test1_result_onnx.jpg', cartoon)
        print('Cartoon portrait has been saved successfully!')


ModuleNotFoundError: ignored

In [ ]:
#照片預處理
#大頭貼切割並轉正
#!/usr/bin/python
!python /content/drive/MyDrive/photo2cartoon/data_process.py --data_path /content/drive/MyDrive/photo2cartoon/image --save_path /content/drive/MyDrive/photo2cartoon/dataset/photo2cartoon/trainA

/usr/local/lib/python3.7/dist-packages/face_alignment/detection/dlib/dlib_detector.py:13: UserWarning: Warning: this detector is deprecated. Please use a different one, i.e.: S3FD.
  warnings.warn('Warning: this detector is deprecated. Please use a different one, i.e.: S3FD.')
Downloading: "https://www.adrianbulat.com/downloads/python-fan/2DFAN4-cd938726ad.zip" to /root/.cache/torch/hub/checkpoints/2DFAN4-cd938726ad.zip
100% 91.9M/91.9M [00:03<00:00, 25.0MB/s]
2022-06-01 17:27:47.089905: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Instructions for updating:
Use tf.gfile.GFile.
  1% 9/968 [00:36<40:45,  2.55s/it]/usr/local/lib/python3.7/dist-packages/face_alignment/api.py:144: UserWarning: No faces were detected.
  warnings.warn("No faces were detected.")
100% 968/968 [30:01<00:00,  1.86s/it]


In [ ]:
#調整當前位置
import os
os.chdir('/content/drive/MyDrive/photo2cartoon')

In [ ]:
#跑訓練
!python /content/drive/MyDrive/photo2cartoon/train.py --dataset /content/drive/MyDrive/photo2cartoon/dataset/photo2cartoon
# --pretrained_weights /content/drive/MyDrive/photo2cartoon/dataset/photo2cartoon_params_latest.pt

##### Information #####
# light :  True
# dataset :  /content/drive/MyDrive/photo2cartoon/dataset/photo2cartoon
# batch_size :  1
# iteration per epoch :  7000
##### Discriminator #####
# discriminator layer :  6

##### Weight #####
# adv_weight :  1
# cycle_weight :  50
# faceid_weight :  1
# identity_weight :  10
# cam_weight :  1000
# rho_clipper:  1.0
# w_clipper:  1.0
training start !
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3704: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
[   10/ 7000] time: 16.8483 d_loss: 3.32426500, g_loss: 1587.87329102
[   20/ 7000] time: 33.7911 d_loss: 2.56982660, g_loss: 1780.30725098
[   30/ 7000] time: 50.9522 d_loss: 1.85234284, g_loss: 1572.38806152
[   40/ 7000] time: 68.6744 d_loss: 2.23051834, g_loss: 1507.71203613
[   50/ 7000] time: 86.5340 d_loss: 1.64969349, g_loss: 1434.56835938
[  

In [ ]:
!python /content/drive/MyDrive/photo2cartoon/test.py --photo_path ./images/test5.jpg --save_path ./images/test5_result_25000.jpg

/usr/local/lib/python3.7/dist-packages/face_alignment/detection/dlib/dlib_detector.py:13: UserWarning: Warning: this detector is deprecated. Please use a different one, i.e.: S3FD.
  warnings.warn('Warning: this detector is deprecated. Please use a different one, i.e.: S3FD.')
Instructions for updating:
Use tf.gfile.GFile.
[Step1: load weights] success!
[Step2: face detect] success!
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3704: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
[Step3: photo to cartoon] success!
Cartoon portrait has been saved successfully!


In [ ]:
while True: pass